<!-- Autogenerated by `scripts/make_examples.py` -->
<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/voxel51/fiftyone-examples/blob/master/examples/GradCam + More Tutorial.ipynb">
            <img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32">
            Try in Google Colab
        </a>
    </td>
    <td>
        <a target="_blank" href="https://nbviewer.jupyter.org/github/voxel51/fiftyone-examples/blob/master/examples/GradCam + More Tutorial.ipynb">
            <img src="https://user-images.githubusercontent.com/25985824/104791634-6efa1d80-5769-11eb-8a4c-71d6cb53ccf0.png" height="32" width="32">
            Share via nbviewer
        </a>
    </td>
    <td>
        <a target="_blank" href="https://github.com/voxel51/fiftyone-examples/blob/master/examples/GradCam + More Tutorial.ipynb">
            <img src="https://user-images.githubusercontent.com/25985824/104791633-6efa1d80-5769-11eb-8ee3-4b2123fe4b66.png" height="32" width="32">
            View on GitHub
        </a>
    </td>
    <td>
        <a href="https://github.com/voxel51/fiftyone-examples/raw/master/examples/GradCam + More Tutorial.ipynb" download>
            <img src="https://user-images.githubusercontent.com/25985824/104792428-60f9cc00-576c-11eb-95a4-5709d803023a.png" height="32" width="32">
            Download notebook
        </a>
    </td>
</table>


# <span style="color:#FF6D04">**GradCam and More with FiftyOne**

## Two Guided Walkthroughs to Help with Model Explainability

## Instance Segmentation Example

###### Imports

In [1]:
import numpy as np
import torch

from pytorch_grad_cam import GradCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad

from pytorch_grad_cam.ablation_layer import AblationLayerVit

### Load from Model Zoo and Dataset Zoo

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset("quickstart")
fo_model = foz.load_zoo_model("deeplabv3-resnet50-coco-torch")

In [2]:
dataset.compute_metadata()
transforms = fo_model.transforms
transforms

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Computing metadata...
 100% |█████████████████| 200/200 [75.8ms elapsed, 0s remaining, 2.6K samples/s] 


Compose(
    Resize(size=520, interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

### Add predictions to dataset

In [5]:
dataset.apply_model(fo_model, label_field="resnet50_seg")

 100% |█████████████████| 200/200 [21.6s elapsed, 0s remaining, 10.9 samples/s]      


![cowboy_predictions](./gradcam_images/cowboy_pred.png)

Sample prediction added with `apply_model`

### We need to wrap the model as to not get a dictionary out, only the mask

In [3]:
classes = fo_model.classes
classes_dict = {value: index for index, value in enumerate(classes)}

In [4]:
from PIL import Image
class ModelWrapper(torch.nn.Module):
    def __init__(self, model): 
        super(ModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, x):
        return self.model(x)["out"]


model = ModelWrapper(fo_model._model)

### Likewise, target needs to be made so that we are targeting the entire mask, but only for the class we are interested in

In [6]:
class SemanticSegmentationTarget:
    def __init__(self, category, mask):
        self.category = category
        self.mask = torch.from_numpy(mask)
        if torch.cuda.is_available():
            self.mask = self.mask.cuda()
        
    def __call__(self, model_output):
        return (model_output[self.category, :, : ] * self.mask).sum()

### Find layers we want to target

In [ ]:
fo_model._model

In [20]:
target_layers = [[model.model.backbone.layer1],
                 [model.model.backbone.layer2],
                 [model.model.backbone.layer3],
                 [model.model.backbone.layer4]]

### Perform GradCam on the model, stepping through the layers of the model to understand how it is looking at an image

#### CAM Methods Available

In [7]:
methods = \
        {"gradcam": GradCAM,
         "scorecam": ScoreCAM,
         "gradcam++": GradCAMPlusPlus,
         "ablationcam": AblationCAM,
         "xgradcam": XGradCAM,
         "eigencam": EigenCAM,
         "eigengradcam": EigenGradCAM,
         "layercam": LayerCAM,}

In [14]:
from skimage.transform import resize



for index, target_layer in enumerate(target_layers):
    cam = methods["gradcam"](model=model,
                    target_layers=target_layers[0],)
    for sample in dataset:

        #Load the Image and Preprocess
        image_path = sample.filepath
        rgb_img = Image.open(image_path)
        input_tensor = transforms(rgb_img).unsqueeze(0).cuda()

        #Generate mask
        output = model(input_tensor)

        #Create the target
        normalized_masks = torch.nn.functional.softmax(output, dim=1).cpu()
        person_mask = normalized_masks[0, :, :, :].argmax(axis=0).detach().cpu().numpy()
        person_category = classes_dict["person"]
        person_mask_float = np.float32(person_mask == person_category)

        targets = [SemanticSegmentationTarget(person_category, person_mask_float)]

        #Perform GradCam
        grayscale_cam = cam(input_tensor=input_tensor,
                            targets=targets,)

        # Here grayscale_cam has only one image in the batch
        grayscale_cam = grayscale_cam[0, :]

        #Save to sample
        sample[f"person_grad_layer_{index+1}"] = fo.Heatmap(map=grayscale_cam)
        sample.save()

### Examples:

Watch the model "ignore" the train and focus on the person

![train_gradcam](./gradcam_images/train_gradcam.gif)

Watch the model

![cowboy_gradcam](./gradcam_images/cowboy_gradcam.gif)

In [16]:
session = fo.launch_app(dataset)

# Vision Transformer Example

## Load the model from model zoo

In [4]:
import os
import eta
import fiftyone.utils.torch as fout
import torchvision

transforms = [fout.ToPILImage(),torchvision.transforms.Resize((224,224)),
             torchvision.transforms.ToTensor(),torchvision.transforms.Normalize(
                    [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]

labels_path = os.path.join(
    eta.constants.RESOURCES_DIR, "imagenet-labels-no-background.txt"
)
transforms = torchvision.transforms.Compose(transforms)

fo_model = fout.load_torch_hub_image_model(
    "facebookresearch/deit:main",
    'deit_tiny_patch16_224',
    hub_kwargs=dict(pretrained=True),
    transforms=transforms,
    output_processor_cls=fout.ClassifierOutputProcessor,
    labels_path=labels_path,
)

Using cache found in /home/dan/.cache/torch/hub/facebookresearch_deit_main
/home/dan/.cache/torch/hub/facebookresearch_deit_main/models.py:63: UserWarning: Overwriting deit_tiny_patch16_224 in registry with models.deit_tiny_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_tiny_patch16_224(pretrained=False, **kwargs):
/home/dan/.cache/torch/hub/facebookresearch_deit_main/models.py:78: UserWarning: Overwriting deit_small_patch16_224 in registry with models.deit_small_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_small_patch16_224(pretrained=False, **kwargs):
/home/dan/.cache/torch/hub/facebookresearch_deit_main/models.py:93: UserWarning: Overwriting deit_base_patch16_224 in registry with models.deit_base_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this 

### Grab Torch Model from FO Models

In [5]:
model = fo_model._model
model.eval()

model = model.cuda()

### We will be targeting the last norm layer in our model for CAM

In [6]:
target_layers = [model.blocks[-1].norm1]

### Load FiftyOne dataset, we will be using the quickstart demo dataset

In [7]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset("quickstart")

Dataset already downloaded
Loading 'quickstart'
 100% |█████████████████| 200/200 [2.2s elapsed, 0s remaining, 90.5 samples/s]       
Dataset 'quickstart' created


In [8]:
dataset.apply_model(fo_model, label_field="deit_class")

 100% |█████████████████| 200/200 [3.0s elapsed, 0s remaining, 163.8 samples/s]      


In [9]:
dataset.compute_metadata()

Computing metadata...
 100% |█████████████████| 200/200 [37.1ms elapsed, 0s remaining, 5.4K samples/s] 


### View model predictions in the app

In [10]:
session=fo.launch_app(dataset)

### To work with vision transformers, we need to reshape the attention tokens to be 2D based on the frame height and width

In [11]:
def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result

## Perform CAM methods on the model

We will use all the available methods too us, targeting whatever the highest performing class is for our model. This will generate a heatmap per sample for each method that show the class activation mapping for the predicted outcome.

In [12]:
from PIL import Image
from skimage.transform import resize
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

for method in methods.keys():
    if method == "ablationcam":
        cam = methods[method](model=model,
                                   target_layers=target_layers,
                                   reshape_transform=reshape_transform,
                                   ablation_layer=AblationLayerVit())
    else:
        cam = methods[method](model=model,
                                   target_layers=target_layers,
                                   reshape_transform=reshape_transform)
    for sample in dataset:
    
        image_path = sample.filepath
        rgb_img = Image.open(image_path)
        input_tensor = transforms(rgb_img).unsqueeze(0)


        #defaults to the highest class
        targets = None

        # AblationCAM and ScoreCAM have batched implementations.
        # You can override the internal batch size for faster computation.
        cam.batch_size = 32

        grayscale_cam = cam(input_tensor=input_tensor,
                            targets=targets,
                            eigen_smooth=True,
                            aug_smooth=True)

        # Here grayscale_cam has only one image in the batch
        grayscale_cam = grayscale_cam[0, :]

        #resized_cam = resize(grayscale_cam, (sample.metadata.height, sample.metadata.width), mode='reflect', anti_aliasing=True,)
        sample[method] = fo.Heatmap(map=grayscale_cam)
        sample.save()

100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 46.14it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 47.65it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 44.69it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 42.99it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 47.80it/s]


...

100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 43.72it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 44.19it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 44.00it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 43.40it/s]


100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 43.75it/s]


In [13]:
session=fo.launch_app(dataset)